In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Dados

In [ ]:
# Importar dados
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSr7tU5tK8cvvR32yypE1PArgXhmNbFJ9bw8w6Sm2zQhyaMs27csoo-77vgFedHw9z25Ez3Qm-geKyU/pub?gid=43084471&single=true&output=csv', skiprows= 1)

df.drop(columns='Unnamed: 0', inplace= True)
df.reset_index(drop = True)
df

### Matriz de correlação

In [ ]:
# Calculando a matriz de correlação
cols = df.columns
string_columns = df.select_dtypes(include=['object']).columns

correlation_matrix = df.drop(columns=string_columns).corr()

plt.figure(figsize=(100, 100))

# Criando um heatmap da matriz de correlação
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')

# Exibindo o gráfico
#plt.show()

# Filtrando pares de colunas com correlação superior a 0.8
# Criando um DataFrame booleano para detectar correlações superiores a 0.8
high_corr_pairs = []

for col1 in correlation_matrix.columns:
    for col2 in correlation_matrix.columns:
        if col1 != col2 and abs(correlation_matrix[col1][col2]) > 0.8:
            high_corr_pairs.append((col1, col2, correlation_matrix[col1][col2]))

# Exibindo os pares de colunas com correlação > 0.8
print("\nPares de colunas com correlação > 0.8:")
for pair in high_corr_pairs:
    print(f"{pair[0]} e {pair[1]}: {pair[2]:.2f}")



Pares de colunas com correlação > 0.8:
Cruise_Mach e Wing_taper_ratio: -0.82
Cruise_Mach e Wing_sweep(deg): 0.83
Wing_taper_ratio e Cruise_Mach: -0.82
Wing_sweep(deg) e Cruise_Mach: 0.83
Wing_sweep(deg) e HT_sweep (deg): 0.96
HT_area (m2) e VT_area (ft2): 0.87
HT_area (m2) e VT_area (m2): 0.87
HT_sweep (deg) e Wing_sweep(deg): 0.96
VT_area (ft2) e HT_area (m2): 0.87
VT_area (ft2) e VT_area (m2): 1.00
VT_area (m2) e HT_area (m2): 0.87
VT_area (m2) e VT_area (ft2): 1.00
Fuselage_length (m) e Main_section_length(m): 0.87
Main_section_length(m) e Fuselage_length (m): 0.87


### Função para fazer os plots

In [ ]:
def plotar(df, coluna1, coluna2):
    """ Plota a relação entre duas variáveis, destacando outliers e ajustando uma regressão linear(sem outliers). """

    df[coluna1] = pd.to_numeric(df[coluna1].astype(str).str.replace(',', '').str.strip(), errors='coerce')
    df[coluna2] = pd.to_numeric(df[coluna2].astype(str).str.replace(',', '').str.strip(), errors='coerce')

    df_valid = df.dropna(subset=[coluna1, coluna2])

    X = df_valid[coluna1].values
    Y = df_valid[coluna2].values

    # Regressão linear inicial para ver outliers
    Z1 = np.polyfit(X, Y, 1)
    p1 = np.poly1d(Z1)
    Y_pred1 = p1(X)

    # Cálculo do erro e desvio padrão para ver os outl
    erros = Y - Y_pred1
    desvio_padrao = np.std(erros)

    # Vamos usar com 1.5
    limite_inferior = Y_pred1 - 1.5 * desvio_padrao
    limite_superior = Y_pred1 + 1.5 * desvio_padrao
    mascara = (Y >= limite_inferior) & (Y <= limite_superior)

    # Separação dos dados filtrados e outliers
    X_filtrado, Y_filtrado = X[mascara], Y[mascara]
    X_outliers, Y_outliers = X[~mascara], Y[~mascara]

    # Regressão linear final com dados filtrados(essa que vamos plotar)
    Z2 = np.polyfit(X_filtrado, Y_filtrado, 1)
    p2 = np.poly1d(Z2)

    plt.figure(figsize=(12, 8))
    # pontos normais e outliers
    plt.scatter(X_filtrado, Y_filtrado, label='Dados Filtrados', color='black')
    plt.scatter(X_outliers, Y_outliers, label='Outliers', color='blue')

    # Regressão linear
    sns.lineplot(x=np.sort(X), y=p2(np.sort(X)), color='gray', linestyle='dotted',
                 label=f'Regressão Linear: f(x) = {p2[0].round(3)} + {p2[1].round(6)} x')

    # Esses deltas são para as labels
    delta_x = (X_filtrado.max() - X_filtrado.min()) / 50
    delta_y = (Y_filtrado.max() - Y_filtrado.min()) / 14

    df_aviao = df_valid[['Parameter', coluna1, coluna2]].copy()
    df_aviao['Y_pred'] = p2(df_aviao[coluna1])
    df_aviao.set_index('Parameter', inplace=True)

    posicoes_rotulos = []
    # se estiver abaixo da reta, a label vai em cima, se estiver em baixo, vai em baixo
    for index, aviao in df_aviao.iterrows():
        x, y, y_pred = aviao[coluna1], aviao[coluna2], aviao['Y_pred']
        label_x, label_y = x - delta_x, y + delta_y * 0.6 if y > y_pred else y - delta_y

        # Isso é para evitar sobreposição de labels
        for px, py in posicoes_rotulos:
            if abs(label_x - px) < delta_x * 1.5 and abs(label_y - py) < delta_y * 1.5:
                label_y += delta_y

        posicoes_rotulos.append((label_x, label_y))
        plt.text(label_x, label_y, str(index), fontsize=16)


    plt.xlabel(coluna1, fontsize=18)
    plt.ylabel(coluna2, fontsize=18)
    plt.legend(fontsize=18)

    ax = plt.gca()
    ax.tick_params(axis='x', top=False, labelsize = 15)
    ax.tick_params(axis='y', right=False, labelsize = 15)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.grid(False)
    plt.show()

In [ ]:
plotar(df, "Number_of_passengers", "MTOW (kg)")

Traceback (most recent call last):
  File "c:\Users\tatyp\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "<string>", line 71, in plotar
  File "C:\Users\tatyp\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\axes\_base.py", line 3389, in tick_params
    self.xaxis.set_tick_params(**xkw)
  File "C:\Users\tatyp\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\axis.py", line 958, in set_tick_params
    kwtrans = self._translate_tick_params(kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tatyp\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\axis.py", line 1102, in _translate_tick_params
    raise ValueError(
ValueError: keyword fontsize is not recognized; valid keywords are ['size', 'width', 'c

In [ ]:
1668.604+435.967279*92

41777.593668


In [ ]:
plotar(df, "Fuselage_diameter (m)", "Fuselage_length (m)")

In [ ]:
# Peso Vazio x MTOW
plotar(df, "MTOW (kg)", "Empty_Weight (kg)")

Relação entre os parâmetros: Empty_Weight (kg) = 32467.473 + -0.121 MTOW (kg)


In [ ]:
# Número de passageiros x MTOW
plotar(df, "Number_of_passengers", "MTOW (kg)")

Relação entre os parâmetros: MTOW (kg) = -211.262 + 460.109 Number_of_passengers


In [ ]:
# Número de passageiros x Comprimento de fuselagem
plotar(df, "Number_of_passengers", "Fuselage_length (m)")

Relação entre os parâmetros: Fuselage_length (m) = 35.278 + -0.043 Number_of_passengers


In [ ]:
# Mach de cruzeiro x Enflechamento da asa
plotar(df, "Cruise_Mach", "Wing_sweep(deg)")

Relação entre os parâmetros: Wing_sweep(deg) = -15.321 + 50.156 Cruise_Mach


In [ ]:
# MTOW x Área da asa
plotar(df, "MTOW (kg)", "Wing_area (ft²)")

Relação entre os parâmetros: Wing_area (ft²) = 16.934 + 0.02 MTOW (kg)


In [ ]:
# MTOW x Range
plotar(df, "MTOW (kg)", "Range (NM)")

Relação entre os parâmetros: Range (NM) = 139.243 + 0.035 MTOW (kg)


In [ ]:
# MTOW x TSFC
plotar(df, "MTOW (kg)", "Total_maximum_thrust (lbf)")


Relação entre os parâmetros: Total_maximum_thrust (lbf) = -3849.936 + 0.731 MTOW (kg)


In [ ]:
# Takeoff_distance x Landing_distance
plotar(df, "Takeoff_distance(m)", "Landing_distance(m)")

Relação entre os parâmetros: Landing_distance(m) = 253.732 + 0.711 Takeoff_distance(m)


In [ ]:
# HT_area x VT_area
plotar(df, "HT_area (m2)", "VT_area (ft2)")

Relação entre os parâmetros: VT_area (ft2) = -6.943 + 8.425 HT_area (m2)
